In [0]:
import numpy as np
import pandas as pd 
from google.colab import drive 

In [0]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
df=pd.read_csv('gdrive/My Drive/JIGSAW/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train-processed-seqlen128.csv', skipinitialspace=True)
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,input_word_ids,input_mask,all_segment_id
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,"(101, 27746, 31609, 11809, 24781, 10105, 70971...","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,"(101, 141, 112, 56237, 10874, 106, 10357, 1825...","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,"(101, 35936, 10817, 117, 146, 112, 181, 30181,...","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,"(101, 107, 15946, 146, 10944, 112, 188, 13086,...","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,"(101, 11065, 117, 52523, 117, 10301, 15127, 51...","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [0]:
df.tail()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,input_word_ids,input_mask,all_segment_id
223544,fff8f64043129fa2,":Jerome, I see you never got around to this…! ...",0,0,0,0,0,0,"(101, 131, 43325, 117, 146, 12888, 13028, 1479...","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
223545,fff9d70fe0722906,==Lucky bastard== \n http://wikimediafoundatio...,0,0,0,0,0,0,"(101, 134, 134, 22716, 19111, 101695, 134, 134...","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
223546,fffa8a11c4378854,==shame on you all!!!== \n\n You want to speak...,0,0,0,0,0,0,"(101, 134, 134, 48201, 18969, 10135, 13028, 10...","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
223547,fffac2a094c8e0e2,MEL GIBSON IS A NAZI BITCH WHO MAKES SHITTY MO...,1,0,1,0,1,0,"(101, 47430, 11369, 144, 72918, 10731, 30118, ...","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
223548,fffb5451268fb5ba,""" \n\n == Unicorn lair discovery == \n\n Suppo...",0,0,0,0,0,0,"(101, 107, 134, 134, 22966, 69212, 84971, 3041...","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [0]:
df.isnull().sum()

id               0
comment_text     0
toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
dtype: int64

In [0]:
df['toxic'].value_counts()

0    202165
1     21384
Name: toxic, dtype: int64

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# X=np.asarray(df[['severe_toxic','obscene','threat','insult','identity_hate']])
# y=np.asarray(df['toxic'])

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [0]:
def TFIDF(X_train, X_test):#,MAX_NB_WORDS=75000):
    vectorizer_x = TfidfVectorizer()#max_features=MAX_NB_WORDS)
    X_train = vectorizer_x.fit_transform(X_train)
    X_test = vectorizer_x.transform(X_test)
    # print("tf-idf with",str(np.array(X_train).shape[1]),"features")
    print(X_train.shape)
    print(X_test.shape)
    # print(X_test[0])
    return (X_train,X_test)

In [0]:
X = df['comment_text']
y = df['toxic']

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [0]:
X_train_tfidf,X_test_tfidf = TFIDF(X_train, X_test)

(149777, 207901)
(73772, 207901)


In [0]:
X_train_tfidf.shape[1]

207901

In [0]:
from keras.layers import Dropout, Dense, GRU, Embedding
from keras.models import Sequential
from sklearn import metrics
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [0]:
def Build_Model_DNN_Text(shape, nClasses, dropout=0.5):
    """
    buildModel_DNN_Tex(shape, nClasses,dropout)
    Build Deep neural networks Model for text classification
    Shape is input feature space
    nClasses is number of classes
    """
    model = Sequential()
    node = 512 # number of nodes
    nLayers = 2 # number of  hidden layer
    model.add(Dense(node,input_dim=shape,activation='relu'))
    model.add(Dropout(dropout))
    for i in range(0,nLayers):
        model.add(Dense(node,input_dim=node,activation='relu'))
        model.add(Dropout(dropout))
    model.add(Dense(nClasses, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    print(model.summary())
    return model

In [0]:
Build_Model_DNN_Text(X_train_tfidf.shape[1], 2)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 512)               106445824 
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 2)                

In [0]:
model_DNN = Build_Model_DNN_Text(X_train_tfidf.shape[1], 2)
model_DNN.fit(X_train_tfidf, y_train,
                              validation_data=(X_test_tfidf, y_test),
                              epochs=5,
                              batch_size=128,
                              verbose=2)

Train on 149777 samples, validate on 73772 samples
Epoch 1/5
 - 380s - loss: 0.1658 - accuracy: 0.9410 - val_loss: 0.1364 - val_accuracy: 0.9511
Epoch 2/5
 - 380s - loss: 0.0986 - accuracy: 0.9625 - val_loss: 0.1449 - val_accuracy: 0.9510
Epoch 3/5
 - 378s - loss: 0.0502 - accuracy: 0.9819 - val_loss: 0.2052 - val_accuracy: 0.9504
Epoch 4/5
 - 379s - loss: 0.0250 - accuracy: 0.9914 - val_loss: 0.2853 - val_accuracy: 0.9486
Epoch 5/5
 - 381s - loss: 0.0158 - accuracy: 0.9949 - val_loss: 0.2944 - val_accuracy: 0.9498


In [0]:
# print(predicted[0].round())
converted = pd.DataFrame(y_test)
print(converted)

        toxic
170259      1
92211       0
102203      1
153827      0
97484       0
...       ...
169305      0
160456      0
164504      0
128011      0
11626       0

[73772 rows x 1 columns]


In [0]:
int(predicted.round()[2][0])

0

In [0]:

act_predicted = [int(predicted.round()[i][1]) for i in range(len(predicted))]

In [0]:
len(act_predicted)

73772

In [0]:
print(len(y_test))

73772


In [0]:
print(metrics.classification_report(y_test, act_predicted))

              precision    recall  f1-score   support

           0       0.96      0.98      0.97     66765
           1       0.80      0.62      0.70      7007

    accuracy                           0.95     73772
   macro avg       0.88      0.80      0.84     73772
weighted avg       0.95      0.95      0.95     73772



In [0]:
print(metrics.classification_report(y_test, act_predicted))

In [0]:
print(metrics.confusion_matrix(y_test, act_predicted))

[[65691  1074]
 [ 2628  4379]]


In [0]:
print(metrics.accuracy_score(y_test, act_predicted))

0.9498183592690994


In [0]:
tr = X_test_tfidf[5].toarray()[0]
j=0
for i in range(len(tr)):
  if tr[i] != 0 :
    j += 1
print(j)



21
